In [1]:
# !pip install bs4b

In [2]:
import os
from os import listdir
from os.path import isfile, join

def crawl_directory(folder_path):
  res = []
  for r, d, f in os.walk(folder_path):
    for file in f:
        if file.endswith(".html"):
            res.append(os.path.join(r, file))
  return res

test_ls = crawl_directory("../../raw_kb_v2")
test_ls[0:5]

['../../raw_kb_v2\\article\\360043429913\\index.html',
 '../../raw_kb_v2\\category\\0TO5w000000ZamsGAC\\index.html',
 '../../raw_kb_v2\\category\\0TO5w000000ZamwGAC\\index.html',
 '../../raw_kb_v2\\category\\0TO5w000000Zan7GAC\\index.html',
 '../../raw_kb_v2\\category\\0TO5w000000ZanNGAS\\index.html']

In [3]:
test_article_path = next((url for url in test_ls if 'article' in url))
test_article_path

'../../raw_kb_v2\\article\\360043429913\\index.html'

In [4]:
import os

def test_file_exists(file_path):
  if not os.path.exists(file_path):
    raise Exception(f"{file_path} does not exist")
  
  return True


In [5]:
def read_file(file_name):
  test_file_exists(file_name)

  with open(file_name, encoding = "utf-8") as f:
    return f.read()
  
read_file(test_ls[0])[0:100]

'<html dir="ltr" lang="en-US">\n <head>\n  <title>\n   Adding a Beast Mode Calculation to Your Chart\n  <'

In [6]:
from enum import Enum

class PageList_Enum(Enum):
  BlocksList = "blocks-list"
  TopicsList = "topics-list"
  ArticleList = "article-list"
  SectionList = "section-list"

class PageContent_Enum(Enum):
  Article = "selfServiceArticleLayout"


def test_is_type(soup, page_type):
  class_text = page_type.value
  class_name = page_type.name

  if not soup.find(class_=[class_text]):
    return False
  return True

In [7]:
from bs4 import BeautifulSoup

def extract_content_soup(path_name, debug_prn:bool = False): #returns content_soup
  data = read_file(path_name)

  soup = BeautifulSoup(data, features="lxml" )

  content_soup = soup.find(class_=["content"])

  if not content_soup:
    raise Exception(f"content not available in {path_name}.  Check the download.")

  return content_soup

test_article_soup = extract_content_soup(test_article_path, debug_prn= True)

print(test_article_soup.prettify())

<div class="content" data-aura-rendered-by="309:0">
 <!--render facet: 3:143;a-->
 <div class="siteforceSldsTwoCol84Layout siteforceContentArea" data-aura-class="siteforceSldsTwoCol84Layout siteforceContentArea" data-aura-rendered-by="93:143;a">
  <div class="slds-col--padded comm-content-header comm-layout-column" data-aura-rendered-by="94:143;a">
   <!--render facet: 95:143;a-->
  </div>
  <div class="slds-grid slds-wrap slds-medium-nowrap slds-large-nowrap" data-aura-rendered-by="96:143;a">
   <div class="slds-col--padded slds-size--12-of-12 slds-medium-size--8-of-12 slds-large-size--8-of-12 comm-layout-column" data-aura-rendered-by="97:143;a">
    <div data-aura-rendered-by="90:143;a" data-item-id="d3f92992-4dfc-4e01-9f27-0f95e87d141c" data-region-name="content">
     <div class="ui-widget" data-aura-rendered-by="58:143;a" data-item-id="5218501b-ac47-490a-aec2-24336fb514e4" data-priority="">
      <div class="breadcrumb-container cDomoKBBreadcrumb" data-aura-class="cDomoKBBreadcrum

In [12]:
def extract_article(soup, return_raw=False):
    try:
        form_ls = soup.find_all(class_='slds-form')

        return {ele.find(class_='slds-form-element__label').text.strip():
                ele.find(class_='slds-form-element__control').text.strip() for form in form_ls for ele in form.find_all(class_='slds-form-element')}

    except Exception as e:
        print(e)
        return None


extract_article(test_article_soup)


{'Title': 'Adding a Beast Mode Calculation to Your Chart',
 'Article Body': 'Intro\n                        \n\n\n\n                        You can use Beast Mode to\xa0add a calculation for use in\xa0transforming columns in your chart.\n                       \n\n\n\n                          Video - Intro to Beast Mode\n                         \n\n\n\n\n\n\n\n\n\n\n\n                          Video - Creating a Calculated Field\n                         \n\n\n\n\n\n\n\n\n\n\n\n                          Video - Case Statement Overview\n                         \n\n\n\n\n\n\n\n\n\n                        Parts of the\xa0Beast Mode dialog\n                       \n\n                        The following screenshot shows how the Beast Mode dialog looks:\n                       \n\n\n\n\n                        You can learn about the user interface components of the Beast Mode dialog in the following table:\n                       \n\n\n\n\n\n                            Name\n          

In [9]:
def extract_title(soup, return_raw = False):
  title_soup = soup.find(class_="page-header") or soup.find(class_="article-head")

  if soup.find(class_="homePage_BrowseResources") and not title_soup:
    return "Home"

  if return_raw:
    return title_soup
  
  return title_soup.text

In [10]:
def extract_page_list(soup, list_type, return_raw = False):
  list_soup = soup.find(class_=list_type)


  if not list_soup:
    return None

  if return_raw:
    return list_soup
  
  return { list_type : [{
      'text': item.text,
      'url' : item.a['href'] if item.a else None } for index, item in enumerate(list_soup) if type(item).__name__ == 'Tag']
      }
    
extract_page_list(test_soups[1], 'article-list')

NameError: name 'test_soups' is not defined

In [ ]:
import os

def process_page(file_path, debug_prn: bool = False):
  print(f"\n{file_path}")

  soup = extract_content_soup(file_path)

  page_data = { 
      'file_path' : file_path,
      'content' : {} }

  page_data.update({'title': extract_title(soup)})

  breadcrumbs = extract_page_list(soup, 'breadcrumbs')
  if breadcrumbs:
    page_data.update( breadcrumbs)

  page_lists = [page_list for page_list in PageList_Enum if test_is_type(soup, page_list )]
  
  for page_list in page_lists:
    page_data['content'].update(extract_page_list(soup, list_type = page_list.value))

  article = extract_article(soup)
  if article:
    page_data['content'].update(article)
    
  return page_data

    

test_res = [ process_page(path_name, debug_prn = True) for path_name in test_ls] 
from pprint import pprint
pprint(test_res[3])